In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import google.generativeai as palm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Configure the API key
palm.configure(api_key='')

In [ ]:
# define a model for embeddings. 
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]

# Select a defined model. 
model = models[0]

In [ ]:
# Define the dataset
positive_data = [
    "I had a great experience at the restaurant last night. The food was delicious!",
    "The customer support team was very helpful. They resolved my issue quickly.",
    "I can't believe how fast my computer is after the recent upgrade.",
    "The movie I watched last weekend was so entertaining. I couldn't stop laughing!",
    "I'm really looking forward to the upcoming vacation. It's been a long year of hard work.",
    "I just finished reading a fantastic novel. The plot was so engaging.",
    "The new smartphone has amazing features. I'm impressed with the camera quality.",
    "The concert last night was incredible. The band played all my favorite songs.",
    "I'm enjoying my new fitness routine. I feel healthier and more energized.",
]

negative_data = [
    "The new smartphone has amazing features, but the battery life is disappointing.",
    "I had a tough day at work today. The workload was overwhelming.",
    "The venue at the concert last night was overcrowded and hot.",
    "I'm not a fan of the new software update. It's causing compatibility issues with my apps.",
    "The service at the restaurant was terrible. The staff was rude, and the food was cold.",
    "I had a terrible experience with the airline. My flight was delayed, and my luggage was lost.",
    "The weather has been awful lately. It's been raining non-stop for days.",
]

neutral_data = [
    "I'm currently reading a new book.",
    "I have some work to do later.",
    "The weather today is neither too hot nor too cold.",
    "I'm planning to go for a walk this evening.",
    "I need to buy groceries on my way home.",
    "I'm considering trying a new restaurant in town.",
    "I watched a documentary last night.",
    "I'm thinking of redecorating my living room.",
]



data = positive_data + negative_data + neutral_data
labels = ['positive'] * len(positive_data) + ['negative'] * len(negative_data) + ['neutral'] * len(neutral_data)



df = pd.DataFrame({'text': data, 'label': labels})

In [ ]:
df.label.unique()

In [ ]:
# split the data into train and text into 80-20 propertion. 
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
def embed_text(text):
    return palm.generate_embeddings(model=model, text=text)['embedding']

# Embed text for training and testing data
X_train_embeddings = np.array([embed_text(text) for text in X_train])
X_test_embeddings = np.array([embed_text(text) for text in X_test])

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
def build_classification_model(input_size, num_classes):
    inputs = keras.Input(shape=(input_size,))
    x = keras.layers.Dense(input_size, activation='relu')(inputs)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

# Create and compile the model
num_classes = len(label_encoder.classes_)
classifier = build_classification_model(X_train_embeddings.shape[1], num_classes)
classifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

# Train the model
NUM_EPOCHS = 20
BATCH_SIZE = 32
history = classifier.fit(X_train_embeddings, y_train_encoded, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,callbacks=[callback], validation_split=0.2)

In [ ]:
def plot_history(history):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(1,2)
  fig.set_size_inches(20, 8)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation'])

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['accuracy'],  label = 'train')
  ax2.plot(history.history['val_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.show()

plot_history(history)

In [ ]:
y_hat = classifier.predict(x=X_test_embeddings)
y_hat = np.argmax(y_hat, axis=1)

In [ ]:
y_hat

In [ ]:
input_text = "This mobile is really giving me lot of trouble"


input_embedding = np.array([embed_text(input_text)])
predicted_probabilities = classifier.predict(input_embedding)
predicted_label = label_encoder.inverse_transform([np.argmax(predicted_probabilities)])[0]
for score in predicted_probabilities:
    score = score.max()
    

# Print the predicted label
print(f"Predicted Label: {predicted_label}, with confidence : {score*100}%")
print(predicted_probabilities)